<a href="https://colab.research.google.com/github/nezihaksu/Airbnb_Istanbul/blob/main/airbnb_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.impute import SimpleImputer
import re

#Explatory Data Analysis

In [104]:
DF = r'/content/drive/MyDrive/listings.csv'
FILE_TYPE = "csv"
IMPUTE = True
ALLOWED_NAN_PERCENTAGE = 10
DROP_KEYWORDS = ["code","zipcode","link","url","id","name","thumbnail","picture","pic","description","note"]
NONE_VALUES = [np.nan,None,"none","null","nan",""," ",0]


class Eda():
  """Explore the dataset."""
  def __init__(self,df,file_type:str):
    if file_type == "xlsx" or  file_type == "xls":
      self.df = pd.read_excel(df,engine="python")
    self.df = pd.read_csv(df,engine="python")
    self.file_type = file_type

  def __call__(self):
    return self.df

  def intro(self):
    return "===INFO===",self.df.info(),"===DESCRIPTION===",self.df.describe(),"===DTYPES==",self.df.dtypes
  
  def unique_values(self):
    #Unique values that are in features.
    for column in self.df.columns:
      print(column.upper()+ " UNIQUE VALUES")
      print(str(df[column].unique())+"\n")

  def missing_values(self):
	  missing_percentage = self.df.isnull().sum()*100/len(self.df)
	  plt.figure(figsize=(5, 15))
	  missing_percentage.plot(kind='barh')
	  plt.xticks(rotation=90, fontsize=10)
	  plt.yticks(fontsize=5)
	  plt.xlabel("Missing Percentage", fontsize=14)
	  plt.show()
   
  #Plotting histograms of the numerical features to see the distribution of each of them.
  def dtype_histogram(self,data_type:str):
    numerical_features = self.df.dtypes[self.df.dtypes == data_type].index.to_list()
    self.df[numerical_features].hist(bins = 50,figsize = (20,15))
    plt.show()

In [105]:
eda = Eda(df=DF,file_type=FILE_TYPE)

In [106]:
class Preprocess():
  """Clean the dataset."""
  def __init__(self,df,file_type:str):
    if file_type == "xlsx" or  file_type == "xls":
      self.df = pd.read_excel(df,engine="python")
    self.df = pd.read_csv(df,engine="python")
    self.file_type = file_type

  #Expanding one column dataframe into multiple columns according to split character.
  def split_column_into_df(self,column_index:int,split_char:str):
    if len(df.columns) == 1:
      quotes_strip = list(self.df.columns)[0].replace(strip_char,'')
      columns_split = quotes_strip.split(split_char)
      self.df = self.df[self.df.iloc[:,0].name].str.split(pat = split_char,expand = True)
      self.df.columns =  columns_split
      self.df.replace(split_char,'',regex = True,inplace = True)
    print("This method is only for explanding single column dataframes!")
    return self.df

  def drop_missing_columns(self,percentage):
    missing_percentage = self.df.isnull().sum()*100/len(self.df)
    features_left = missing_percentage[missing_percentage < percentage].index
    return self.df[features_left]

  #Drop columns by their names.
  def drop_column_contains(self,keywords:list):
    for keyword in keywords:
      keyword_pattern = re.compile(keyword)
      for column in self.df.columns:
        if keyword_pattern.search(column):
          self.df.drop(column,axis=1,inplace=True)
    return self.df

  #Loops over the values of the column until hitting the specified limit to find letter character to match and drop the column.
  def _drop_value_type(self,pattern,search_limit:int):
      for column in self.df.columns:
        for value in self.df[column].values[:search_limit]:
          if pattern.match(str(value)):
            print(column)
            self.df.drop(column,axis=1,inplace=True)

      return self.df

  def drop_text_columns(df,index):
  pat1 = r'[A-z][A-z]+?\W'
  pat2 = r'[A-z][A-z]+?://'
  letter_pattern = re.compile(r'|'.join((pat1,pat2)))
  for column in df.columns:
    value = df[column].values[index]
    if letter_pattern.match(str(value)) and value not in none_values:
      df.drop(column,axis=1,inplace=True)
  
  return df
    
  def drop_date_columns(self,search_limit:int):
    date_pattern = re.compile(r'([12]\d{3}/|-|.|,(0[1-9]|1[0-2])/|-|.|,(0[1-9]|[12]\d|3[01]))')
    return self._drop_value_type(date_pattern,search_limit)
  
  def impute_nan_values(self,strategy="most_frequent"):
    simple_imputer = SimpleImputer(strategy=strategy)
    for column in df.columns:
      if pd.DataFrame.any(df[column].isnull()):
        df[column] = simple_imputer.fit_transform(df[column])
       


In [107]:
preprocess = Preprocess(DF,FILE_TYPE)

In [108]:
df = pd.read_csv(DF,engine="python")
df

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,4826,https://www.airbnb.com/rooms/4826,20190427155359,2019-04-28,The Place,My place is close to great views. My place is ...,A double bed apartment on the Asian side of Is...,My place is close to great views. My place is ...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/15347/ba87...,NaN,6603,https://www.airbnb.com/users/show/6603,Kaan,2009-01-14,"Istanbul, Istanbul, Turkey","Hello...\r\nI am Kaan and I am from Istanbul, ...",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/6603/profile_...,https://a0.muscache.com/im/users/6603/profile_...,Üsküdar,1.0,1.0,"['email', 'phone', 'facebook', 'reviews']",t,f,"Istanbul Province, Istanbul, Turkey",Üsküdar,Uskudar,NaN,Istanbul Province,Istanbul,34684,Istanbul,"Istanbul Province, Turkey",TR,Turkey,41.05650,29.05367,f,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Smoking ...",700.0,$624.00,NaN,NaN,NaN,NaN,2,$154.00,1,730,1,1,730,730,1.0,730.0,24 months ago,t,30,60,90,365,2019-04-28,1,0,2009-06-01,2009-06-01,100.0,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,flexible,f,f,1,1,0,0,0.01
1,20815,https://www.airbnb.com/rooms/20815,20190427155359,2019-04-28,The Bosphorus from The Comfy Hill,Watch The Bosphorus from The Comfy Hill. A spa...,"The low green hills of Arnavutköy, the lovely,...",Watch The Bosphorus from The Comfy Hill. A spa...,none,"The lovely neighborhood, Arnavutkoy, is locate...",The house may be suited with extra inflatable ...,"The city center, Taksim is about 6-7 kilometer...",Our dear guests may enjoy the oven and the was...,"Depending on our time available, we may even t...",- Windows facing the street should not be left...,NaN,NaN,https://a0.muscache.com/im/pictures/94805791/4...,NaN,78838,https://www.airbnb.com/users/show/78838,Gülder,2010-02-08,"Ankara, Ankara, Turkey",Başar Family\r\n\r\nGülder (mother): Owner of ...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/78838/profile...,https://a0.muscache.com/im/users/78838/profile...,Beşiktaş,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'wor...",t,f,"Istanbul, Turkey",Beşiktaş,Besiktas,NaN,Istanbul,NaN,34345,Istanbul,"Istanbul, Turkey",TR,Turkey,41.06984,29.04545,t,Apartment,Entire home/apt,3,1.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",""Air conditioning"",

In [178]:
def drop_text_columns(df,index):
  pat1 = r'[A-z][A-z]+?\W'
  pat2 = r'[A-z][A-z]+?://'
  letter_pattern = re.compile(r'|'.join((pat1,pat2)))
  for column in df.columns:
    value = df[column].values[index]
    if letter_pattern.match(str(value)) and value not in none_values:
      df.drop(column,axis=1,inplace=True)
  
  return df

drop_text_columns(df)

,id,scrape_id,last_scraped,experiences_offered,house_rules,thumbnail_url,medium_url,xl_picture_url,host_id,host_name,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,state,zipcode,market,country_code,country,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,4826,20190427155359,2019-04-28,none,NaN,NaN,NaN,NaN,6603,Kaan,2009-01-14,NaN,NaN,NaN,f,Üsküdar,1.0,1.0,"['email', 'phone', 'facebook', 'reviews']",t,f,Üsküdar,Uskudar,NaN,Istanbul,34684,Istanbul,TR,Turkey,41.05650,29.05367,f,Apartment,2,1.0,0.0,1.0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Smoking ...",700.0,$624.00,NaN,NaN,NaN,NaN,2,$154.00,1,730,1,1,730,730,1.0,730.0,24 months ago,t,30,60,90,365,2019-04-28,1,0,2009-06-01,2009-06-01,100.0,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,flexible,f,f,1,1,0,0,0.01
1,20815,20190427155359,2019-04-28,none,- Windows facing the street should not be left...,NaN,NaN,NaN,78838,Gülder,2010-02-08,NaN,NaN,NaN,f,Beşiktaş,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'wor...",t,f,Beşiktaş,Besiktas,NaN,NaN,34345,Istanbul,TR,Turkey,41.06984,29.04545,t,Apartment,3,1.0,2.0,2.0,"{TV,""Cable TV"",""Air conditioning"",Kitchen,""Fre...",NaN,$113.00,"$1,348.00","$4,615.00",$588.00,NaN,4,$208.00,30,210,30,30,210,210,30.0,210.0,3 months ago,t,0,0,0,117,2019-04-28,41,5,2010-03-24,2018-11-07,90.0,9.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,moderate,t,f,2,1,1,0,0.37
2,25436,20190427155359,2019-04-27,none,NaN,NaN,NaN,NaN,105823,Yesim,2010-04-09,NaN,NaN,NaN,f,Beşiktaş,1.0,1.0,"['email', 'phone', 'offline_government_id', 's...",t,f,Beşiktaş,Besiktas,NaN,İstanbul,34400,Istanbul,TR,Turkey,41.07731,29.03891,t,Serviced apartment,6,1.0,2.0,5.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",969.0,$232.00,"$2,980.00","$9,934.00","$19,869.00",$0.00,1,$0.00,21,1125,21,21,1125,1125,21.0,1125.0,2 weeks ago,t,0,0,0,2,2019-04-27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,t,f,strict_14_with_grace_period,f,f,1,1,0,0,NaN
3,27271,20190427155359,2019-04-27,none,This is the place I am living. Not like the pl...,NaN,NaN,NaN,117026,Mutlu,2010-05-02,NaN,NaN,NaN,f,Cihangir,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,Cihangir,Beyoglu,NaN,NaN,34433,Istanbul,TR,Turkey,41.03220,28.98216,f,Apartment,2,1.0,1.0,1.0,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",NaN,$267.00,"$1,523.00","$5,431.00",$662.00,$265.00,2,$99.00,5,90,5,5,90,90,5.0,90.0,3 months ago,t,0,8,8,277,2019-04-27,2,1,2014-10-20,2018-05-04,100.0,10.0,9.0,10.0,10.0,10.0,10.0,f,NaN,NaN,f,f,moderate,f,f,1,1,0,0,0.04
4,28277,20190427155359,2019-04-27,none,NaN,NaN,NaN,NaN,121607,Alen,2010-05-10,within an hour,100%,NaN,f,Şişli,19.0,19.0,"['email', 'phone', 'reviews']",t,f,Şişli,Sisli,NaN,NaN,34373,Istanbul,TR,Turkey,41.04471,28.98567,t,Serviced apartment,5,1.0,1